In [1]:
import pandas as pd
import numpy as np
from fuzzywuzzy import process, fuzz
import pickle
import re
import datetime

# Data Cleaning

### Betfair odds (dict_betfair) 

In [2]:
#pickle data scraped in section 1
dict_betfair = pickle.load(open('dict_betfair', 'rb'))
#initialize storage (we'll use these dictionaries to match names between betfair and historical_data)
dict_home_name_matching = {}
dict_away_name_matching = {}
#fill the dictionary with a list of names of all home and away teams that will play during the week
for league in dict_betfair:
    dict_betfair[league][['home_team', 'away_team']] = dict_betfair[league]['Teams'].str.extract(r'(.+)\n(.+)')
    dict_home_name_matching[league] = dict_betfair[league].groupby('home_team', as_index=False).count()[['home_team']]
    dict_away_name_matching[league] = dict_betfair[league].groupby('away_team', as_index=False).count()[['away_team']]

### Historical Data (dict_historical_data)

In [3]:
#we create a dictionary dict_countries similar to the one we used in the scraping section 
#but in this case we need to specify the names considered in the link to download the csv e.g. "SP1" (Spanish League)
dict_countries = {
              'Spanish La Liga':'SP1', 'Spanish Segunda Division':'SP2',
              'German Bundesliga':'D1', 
              'German Bundesliga 2':'D2',
              'Italian Serie A':'I1', 
              'Italian Serie B':'I2',
              'English Premier League':'E0', 'English League 1':'E2', 'English League 2':'E3',
              'French Ligue 1': 'F1', 'French Ligue 2':'F2',
              'Dutch Eredivisie':'N1',
              'Belgian First Division A':'B1',
              'Portuguese Primeira Liga':'P1',
              'Turkish Super League':'T1',
              'Greek Super League':'G1',
             }

#dict_historical_data contains data of the past 5 years. we'll use it to manage 2 dataframes: df_historical_data and df_profile
dict_historical_data = {} 

#to download all the leagues we loop through the dictionary
for league in dict_countries:
    frames = []
    for i in range(15, 21):
        try:
            df = pd.read_csv("http://www.football-data.co.uk/mmz4281/"+str(i)+str(i+1)+"/"+dict_countries[league]+".csv")
        except: #Italian Serie B (0xa0 utf-8)
            df = pd.read_csv("http://www.football-data.co.uk/mmz4281/"+str(i)+str(i+1)+"/"+dict_countries[league]+".csv", encoding='unicode_escape')
        df = df.assign(season=i)
        frames.append(df)
    df_frames = pd.concat(frames)
    df_frames = df_frames.rename(columns={'Date':'date', 'HomeTeam':'home_team', 'AwayTeam':'away_team',
                        'FTHG': 'home_goals', 'FTAG': 'away_goals'})
    dict_historical_data[league] = df_frames

In [4]:
## Upload (..verify if uploaded correctly)
# output = open('dict_historical_data', 'wb') #don't forget to change name_of_file
# pickle.dump(dict_historical_data, output)
# output.close()

# downloading series script
input_file = open('dict_historical_data','rb')
dict_historical_data = pickle.load(input_file)

### Matching team names. Replacing "Historical Data" team names in  betfair dataframes

In [5]:
#loop through each league
for league in dict_historical_data:
    #picking unique team names inside the historical_data team names
    all_teams = dict_historical_data[league]['home_team'].unique().tolist()
    #matching betfair names (dict_betfair -> dict_home_name_matching, dict_away_name_matching) with historical data (dict_historical_data -> all_teams)
    dict_home_name_matching[league][['teams_matched', 'score']] = dict_home_name_matching[league]['home_team'].apply(lambda x:process.extractOne(x, all_teams, scorer=fuzz.token_set_ratio)).apply(pd.Series)
    dict_away_name_matching[league][['teams_matched', 'score']] = dict_away_name_matching[league]['away_team'].apply(lambda x:process.extractOne(x, all_teams, scorer=fuzz.token_set_ratio)).apply(pd.Series)
    #Replacing "Historical Data" team names (teams_matched) in  betfair dataframes
    home_teams = pd.merge(dict_betfair[league], dict_home_name_matching[league], on='home_team',
                          how='left')[['Dates', 'over2.5', 'btts', 'teams_matched']].rename(columns={'teams_matched':'home_team'})
    away_teams = pd.merge(dict_betfair[league], dict_away_name_matching[league], on='away_team',
                         how='left')[['teams_matched']].rename(columns={'teams_matched':'away_team'})
    #updating values
    dict_betfair.update({league:pd.concat([home_teams, away_teams], axis=1)})

# Creating Profiles to calculate Real odds and Stats

In [6]:
#df_historical_data: 5 years for h2h analysis (it doesn't have 'HST', 'AST', 'HC', 'AC') Only recent seasons have all columns available
df_historical_data = pd.concat(dict_historical_data, ignore_index=True)
df_historical_data = df_historical_data[['date', 'home_team', 'away_team', 'home_goals', 'away_goals', 'season']] 

In [7]:
#df_profile: 2 years (current + last season) for real odds and stats calculation (it has 'HST', 'AST', 'HC', 'AC')
#You can add or omit seasons    
seasons = ['19', '20']

df_profile = pd.concat(dict_historical_data, ignore_index=True)
df_profile = df_profile[df_profile['season'].isin(seasons)]
df_profile = df_profile[['date', 'home_team', 'away_team', 'home_goals', 'away_goals',
                               'HST', 'AST', 'HC', 'AC', 'season']]
df_profile = df_profile.rename(columns={'HST':'home_shots_target', 'AST':'away_shots_target',
                                              'HC':'home_corners', 'AC':'away_corners'})

df_profile

,date,home_team,away_team,home_goals,away_goals,home_shots_target,away_shots_target,home_corners,away_corners,season
1520,16/08/2019,Ath Bilbao,Barcelona,1.0,0.0,5.0,2.0,3.0,8.0,19
1521,17/08/2019,Celta,Real Madrid,1.0,3.0,4.0,11.0,6.0,4.0,19
1522,17/08/2019,Valencia,Sociedad,1.0,1.0,6.0,3.0,3.0,3.0,19
1523,17/08/2019,Mallorca,Eibar,2.0,1.0,4.0,5.0,9.0,3.0,19
1524,17/08/2019,Leganes,Osasuna,0.0,1.0,2.0,2.0,8.0,0.0,19
...,...,...,...,...,...,...,...,...,...,...
32515,07/02/2021,PAOK,Apollon,2.0,2.0,7.0,2.0,15.0,2.0,20
32516,07/02/2021,Olympiakos,OFI Crete,3.0,0.0,7.0,2.0,7.0,1.0,20
32517,07/02/2021,Volos NFC,Asteras Tripolis,0.0,1.0,3.0,2.0,3.0,0.0,20
32518,07/02/2021,AEK,Aris,0.0,2.0,3.0,4.0,4.0,3.0,20


# Calculating Stats

In [8]:
#My method
# df_stats = pd.concat([
#     df_profile.groupby('home_team').mean().round(2)[['home_goals', 'home_shots_target', 'home_corners']],
#     df_profile.groupby('away_team').mean().round(2)[['away_goals', 'away_shots_target', 'away_corners']]],
#     axis=1)

#Betpractice method for average goals
df_home = df_profile.groupby('home_team').mean().round(2)
df_home['home_goals_betpractice'] = df_home['home_goals'] + df_home['away_goals']
df_away = df_profile.groupby('away_team').mean().round(2)
df_away['away_goals_betpractice'] = df_away['home_goals'] + df_away['away_goals']

df_stats = pd.concat([
    df_home[['home_goals_betpractice', 'home_shots_target', 'home_corners']],
    df_away[['away_goals_betpractice', 'away_shots_target', 'away_corners']],
], axis=1)

df_stats.rename(columns={'home_goals_betpractice':'home_goals', 'away_goals_betpractice':'away_goals'},
               inplace=True)
df_stats

,home_goals,home_shots_target,home_corners,away_goals,away_shots_target,away_corners
AEK,2.79,5.14,6.00,2.32,4.79,4.18
AFC Wimbledon,2.51,3.55,5.34,2.93,3.50,4.12
AZ Alkmaar,3.13,6.39,7.70,3.26,5.48,5.39
Accrington,2.87,4.83,5.63,2.55,4.00,5.28
Ajaccio,1.93,3.65,5.04,2.27,3.69,4.12
...,...,...,...,...,...,...
Wycombe,3.00,4.61,4.83,1.94,3.19,4.69
Xanthi,1.82,3.31,4.94,2.00,2.41,2.65
Yeni Malatyaspor,2.48,4.00,4.72,2.93,4.39,4.00
Zaragoza,2.42,4.42,4.85,2.18,3.18,4.12


# Creating Market Filters

### Filter 1 (Date) and Filter 2 (Stats/Presets) (run from here)

###### Note: The following error "You are trying to merge on object and float64 columns. If you wish to proceed you should use pd.concat" happens when df_betfair doesn't have data available for the date you're selecting (the data scrape only have games for the next 4 days or so)

In [9]:
#inputs
match_date = '2021-02-14'
dict_filter = {'goals':2.5, 'shots_target':9, 'corners':9}
#-----------------------------------
#concatenate all leagues in dict_betfair
df_betfair = pd.concat(dict_betfair, ignore_index=True)
df_betfair = df_betfair[df_betfair['Dates']==match_date]
#split home and away teams
home = df_betfair['home_team'].to_list()
away = df_betfair['away_team'].to_list()

goals = []
shots_target = []
corners = []
promoted_team = [] #no stats available (promoted teams that don't have stats available in the recent 2 years)
#calculating and storing stats (average goals, total shots on target and total corners)
for home_team, away_team in zip(home, away):
    try:
        goals.append(round((df_stats.loc[home_team, 'home_goals'] + df_stats.loc[away_team, 'away_goals'])/2, 2))#betpractice method
        shots_target.append(df_stats.loc[home_team, 'home_shots_target'] + df_stats.loc[away_team, 'away_shots_target'])
        corners.append(df_stats.loc[home_team, 'home_corners'] + df_stats.loc[away_team, 'away_corners'])       
    except:
        print('Possible promoted teams: '+home_team+' - '+away_team) #no stats data
        goals.append(0)
        shots_target.append(0)
        corners.append(0)

#filtering games out            
df_filters = pd.DataFrame.from_dict({'home_team':home, 'away_team':away, 'goals':goals,
                                     'shorts_target':shots_target, 'corners':corners})
df_filters = df_filters[(df_filters['goals']>=dict_filter['goals']) &
                        (df_filters['shorts_target']>=dict_filter['shots_target']) &
                        (df_filters['corners']>=dict_filter['corners'])]
df_filters = pd.merge(df_betfair, df_filters, on=['home_team', 'away_team'])

Possible promoted teams: Goztep - Eskisehirspor


### Real Odds

In [10]:
def calculate_real_odds(home_team, away_team, df_profile, market):
    """Calculates the reals odds in a match with data based on selected seasons (df_profile)"""
    df_profile = df_profile.assign(total_goals=df_profile['home_goals'] + df_profile['away_goals'])
    df_profile['Over/Under'] = np.where(df_profile['total_goals']>2, 'Over 2.5', 'Under 2.5')
    df_profile['BTTS'] = np.where((df_profile['home_goals']>0) & (df_profile['away_goals']>0), True, False)
    dict_markets = {'BTTS':[True, False], 'Over/Under':['Over 2.5', 'Under 2.5']}
    df_real_odds = df_profile[(df_profile['home_team']==home_team)|(df_profile['away_team']==away_team)].groupby(market).count()[['total_goals']]
    option1 = df_real_odds.loc[dict_markets[market][0], 'total_goals']#over2.5
    option2 = df_real_odds.loc[dict_markets[market][1], 'total_goals']#under2.5
    percentage_odds_over = option1 / (option1 + option2)
    real_odds = round(1/percentage_odds_over, 2)
    return real_odds

### Filter 3: Real Odds: BTTS, Over 2.5 <= 2

In [11]:
#inputs
dict_filter_real_odds = {'real_odds_over':2, 'real_odds_btts':2} #at least 50% of chances
#-------
#simplifying formula
calculate_over = lambda x, y : calculate_real_odds(x, y, df_profile, 'Over/Under')
calculate_btts = lambda x, y : calculate_real_odds(x, y, df_profile, 'BTTS')
#aplying formula to calculate reald odds
df_filters['real_odds_over'] = df_filters.apply(lambda x:calculate_over(x['home_team'], x['away_team']), axis=1)
df_filters['real_odds_btts'] = df_filters.apply(lambda x:calculate_btts(x['home_team'], x['away_team']), axis=1)
#filtering teams out based on reald odds
df_filters = df_filters[(df_filters['real_odds_over']<=dict_filter_real_odds['real_odds_over']) &
                        (df_filters['real_odds_btts']<=dict_filter_real_odds['real_odds_btts'])]

# Finding Value Bets

In [12]:
#picking columns
df_system = df_filters[['home_team', 'away_team', 'goals', 'shorts_target', 'corners',
                      'real_odds_over', 'over2.5', 'real_odds_btts', 'btts']]
#renaming columns
df_system.rename(columns={'shorts_target':'SoT', 'corners':'C', 'goals':'G', 'over2.5':'OVER',
                          'btts':'BTTS', 'real_odds_over':'RO_OVER', 'real_odds_btts':'RO_BTTS'}, inplace=True)
#extracting over2.5 and btts=True values
df_system = df_system.assign(OVER = df_system['OVER'].str.extract(r'(.+)\n.+').astype(float))
df_system = df_system.assign(BTTS = df_system['BTTS'].str.extract(r'(.+)\n.+').astype(float))
#calculating value of bets
df_system['V_OVER'] = df_system['OVER'] - df_system['RO_OVER']
df_system['V_BTTS'] = df_system['BTTS'] - df_system['RO_BTTS']
#sorting matches by value
df_system.sort_values('V_OVER', ascending=False)

/Applications/anaconda3/envs/scrapping/lib/python3.7/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,home_team,away_team,G,SoT,C,RO_OVER,OVER,RO_BTTS,BTTS,V_OVER,V_BTTS
7,Sampdoria,Fiorentina,3.02,10.86,10.83,1.46,1.75,1.63,1.60,0.29,-0.03
5,Inter,Lazio,3.26,13.13,11.38,1.42,1.65,1.46,1.62,0.23,0.16
14,Beerschot VA,Mechelen,3.26,9.27,9.65,1.50,1.70,1.62,1.58,0.20,-0.04
9,Arsenal,Leeds,3.38,9.75,12.12,1.46,1.58,1.71,1.52,0.12,-0.19
10,Monaco,Lorient,3.16,9.11,10.11,1.46,1.55,1.76,1.70,0.09,-0.06
6,Roma,Udinese,3.06,12.08,12.92,1.61,1.70,1.61,1.68,0.09,0.07
13,Vitesse,Twente,2.76,9.30,9.74,1.61,1.68,1.73,1.65,0.07,-0.08
0,Ein Frankfurt,FC Koln,3.38,9.67,11.66,1.49,1.55,1.41,1.65,0.06,0.24
1,Wolfsburg,M'gladbach,2.84,10.38,10.73,1.70,1.62,1.70,1.50,-0.08,-0.20
15,Standard,Antwerp,2.63,9.46,10.14,1.83,1.75,1.67,1.62,-0.08,-0.05


# Match Analysis

### Hand to Hand (5 years)

In [13]:
#introduce index of table above
index = 5 #example inter - lazio (2021-02-14)
#-------
home_h2h = df_system.loc[index, 'home_team']
away_h2h = df_system.loc[index, 'away_team']
teams_h2h = [home_h2h, away_h2h]

df_h2h = df_historical_data[(df_historical_data['home_team'].isin(teams_h2h)) & 
           (df_historical_data['away_team'].isin(teams_h2h))]
df_h2h[['date',	'home_team', 'away_team', 'home_goals', 'away_goals']]

,date,home_team,away_team,home_goals,away_goals
8271,20/12/15,Inter,Lazio,1.0,2.0
8461,01/05/16,Lazio,Inter,2.0,0.0
8659,21/12/16,Inter,Lazio,3.0,0.0
8853,21/05/17,Lazio,Inter,1.0,3.0
9051,30/12/17,Inter,Lazio,0.0,0.0
9242,20/05/18,Lazio,Inter,2.0,3.0
9346,29/10/2018,Lazio,Inter,0.0,3.0
9534,31/03/2019,Inter,Lazio,0.0,1.0
9673,25/09/2019,Inter,Lazio,1.0,0.0
9866,16/02/2020,Lazio,Inter,2.0,1.0


## Current season analysis

In [14]:
current_season = max(df_profile['season'].to_list())
df_current_season = df_profile[df_profile['season']==current_season]
df_current_season

,date,home_team,away_team,home_goals,away_goals,home_shots_target,away_shots_target,home_corners,away_corners,season
1900,12/09/2020,Eibar,Celta,0.0,0.0,1.0,3.0,3.0,3.0,20
1901,12/09/2020,Granada,Ath Bilbao,2.0,0.0,2.0,2.0,1.0,2.0,20
1902,12/09/2020,Cadiz,Osasuna,0.0,2.0,3.0,4.0,6.0,2.0,20
1903,13/09/2020,Alaves,Betis,0.0,1.0,2.0,4.0,6.0,4.0,20
1904,13/09/2020,Valladolid,Sociedad,1.0,1.0,3.0,2.0,5.0,3.0,20
...,...,...,...,...,...,...,...,...,...,...
32515,07/02/2021,PAOK,Apollon,2.0,2.0,7.0,2.0,15.0,2.0,20
32516,07/02/2021,Olympiakos,OFI Crete,3.0,0.0,7.0,2.0,7.0,1.0,20
32517,07/02/2021,Volos NFC,Asteras Tripolis,0.0,1.0,3.0,2.0,3.0,0.0,20
32518,07/02/2021,AEK,Aris,0.0,2.0,3.0,4.0,4.0,3.0,20


### Past 5 games home and away in current season (h2h)

In [15]:
#last 5
pd.concat([df_current_season[df_current_season['home_team']==home_h2h][-5:], 
           df_current_season[df_current_season['away_team']==home_h2h][-5:]])

pd.concat([df_current_season[df_current_season['home_team']==away_h2h][-5:], 
           df_current_season[df_current_season['away_team']==away_h2h][-5:]])

,date,home_team,away_team,home_goals,away_goals,home_shots_target,away_shots_target,home_corners,away_corners,season
10135,20/12/2020,Lazio,Napoli,2.0,0.0,2.0,6.0,4.0,9.0,20
10160,06/01/2021,Lazio,Fiorentina,2.0,1.0,6.0,8.0,2.0,9.0,20
10176,15/01/2021,Lazio,Roma,3.0,0.0,9.0,2.0,1.0,7.0,20
10195,24/01/2021,Lazio,Sassuolo,2.0,1.0,8.0,1.0,9.0,6.0,20
10216,07/02/2021,Lazio,Cagliari,1.0,0.0,6.0,3.0,12.0,2.0,20
10117,15/12/2020,Benevento,Lazio,1.0,1.0,6.0,3.0,4.0,8.0,20
10140,23/12/2020,Milan,Lazio,3.0,2.0,5.0,9.0,4.0,6.0,20
10150,03/01/2021,Genoa,Lazio,1.0,1.0,3.0,4.0,4.0,9.0,20
10170,10/01/2021,Parma,Lazio,0.0,2.0,3.0,5.0,5.0,6.0,20
10202,31/01/2021,Atalanta,Lazio,1.0,3.0,4.0,7.0,2.0,2.0,20


### Goal average in current season (h2h)

In [16]:
def calculate_current_season_average_goals(team):
    #goals scored when playing home
    a = df_current_season[df_current_season['home_team']==team]['home_goals'].mean().round(2)
    #goals conceded when playing home
    b = df_current_season[df_current_season['away_team']==team]['away_goals'].mean().round(2)

    #goals scored when playing away
    c = df_current_season[df_current_season['home_team']==team]['away_goals'].mean().round(2)
    #goals conceded when playing away
    d = df_current_season[df_current_season['away_team']==team]['home_goals'].mean().round(2)
    return pd.DataFrame({'home':[a, c], 'away':[b, d]}, index=['Scored', 'Conceded'])

calculate_current_season_average_goals(away_h2h) #home_h2h(Inter), away_h2h(Lazio)

,home,away
Scored,1.55,1.9
Conceded,1.27,1.3
